In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
import matplotlib.pyplot as plt  
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 500
mpl.rcParams['font.family']= 'Arial'

In [3]:
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats

#my_colormap = LinearSegmentedColormap.from_list("", [ "darkred","white"])
my_colormap = LinearSegmentedColormap.from_list("", ['darkred',"white", "lightsalmon"])
my_colormap1 = LinearSegmentedColormap.from_list("", ['#735c12',"white", "#922b05"])
my_colormap2 = LinearSegmentedColormap.from_list("", ['steelblue',"white", "darkred"])

In [839]:
color_dict = {
'Adenoviridae':'#007440',
'Herpesviridae':'#789C49',
'Papillomaviridae':'#5AA987',
'Polyomaviridae':'#ACC532',
'Poxviridae':'#B8C98D',
'Reoviridae':'#E8DB00',
'Anelloviridae':'#F5D19E',
'Parvoviridae':'#93583A',
'Retroviridae':'#919191',
'Caliciviridae':'#496E93',
'Coronaviridae':'#9DCAE6',
'Flaviviridae':'#BFCBF1',
'Hepeviridae':'#8080EB',
'Picornaviridae':'#7D89C1',
'Togaviridae':'#47499C',
'Bornaviridae':'#522B55',
'Filoviridae':'#955B8E',
'Kolmioviridae':'#AA97BC',
'Orthomyxoviridae':'#F6E2DC',
'Paramyxoviridae':'#F4C8C5',
'Phenuiviridae':'#E48180',
'Pneumoviridae':'#C24F4F',
'Rhabdoviridae':'#8A292D',
'palegreen':'palegreen',
'darkorange':'darkorange',
'black':'black',
'white':'white',
'yellow':'yellow',
'red':'red',
'cyan':'cyan',
'purple':'purple'
}

all_colors = list(color_dict.values())

colors_dict = dict(zip(list(range(30)),all_colors))

In [ ]:
SAVE_ROOT = '/home/lyz/PhaSepGAT/saves/INTEGRATE_ProSE80d_pos1neg1/'
preds = pd.read_pickle(f'{SAVE_ROOT}preds.pkl')
genes = pd.read_pickle(f'{SAVE_ROOT}genes.pkl')

In [841]:
genename_to_uniprot = pd.read_pickle("../data/HUMAN_9606_idmapping_genename.pkl")
genename_to_uniprot_integrate = genename_to_uniprot.set_index(['Uniprot']).loc[list(set(genes)&set(genename_to_uniprot['Uniprot'].tolist())),:].reset_index()

genename_to_uniprot_dict = genename_to_uniprot_integrate.set_index(['ID']).to_dict()['Uniprot']
uniprot_to_genename_dict = genename_to_uniprot_integrate.set_index(['Uniprot']).to_dict()['ID']

In [842]:
llps_scores = pd.read_pickle("../data/HUMAN_9606_PhaSePred_SaPS_PdPS.pkl")
llps_scores.head(2)
llps_dict = llps_scores.set_index(['entry']).to_dict()

In [843]:
entry2pdps_dict = llps_scores.set_index('entry').to_dict()['PdPS-10fea']
entry2saps_dict = llps_scores.set_index('entry').to_dict()['SaPS-10fea']

In [844]:
integrate = pd.read_csv("../data/Interactome-Datasets/integrate.csv")
integrate.head(2)

,A,B
0,Q9H2S6,Q9NPE6
1,Q9H2S6,Q9BXK5


# Read Enrichment results

In [ ]:
cluster_features_enrich = pd.read_pickle("/home/lyz/PhaSepGAT/saves/INTEGRATE_ProSE80d_pos1neg1/PS_clusters_allfeature_enrichment.pkl")
cluster_features_enrich.head(1)

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,Frac RHK_IVMALP_deltazscores,Frac RHK_FWY_deltazscores,Frac STNQCG_IVMALP_deltazscores,Frac STNQCG_FWY_deltazscores,Frac IVMALP_FWY_deltazscores,Frac WT_pos_block,Frac WT_neg_block,Frac WT_all_block,Frac Hydropathy,Frac NCPR
0,1,31,0.3021,140.5,758.1,0.1463,0.999874,Q9Y5B9 Q08945 Q96P70 Q9BTT0 P62805 Q01831 P499...,0.001036,0.580645,...,0.064516,0.193548,0.064516,0.064516,0.032258,0.451613,0.387097,0.354839,0.0,0.483871


In [914]:
cluster_features_enrich[cluster_features_enrich['Cluster']==2069]

,Cluster,Size,Density,Internal weight,External weight,Quality,P-value,Members,FDR PS proteins,Frac PS proteins,...,Frac RHK_IVMALP_deltazscores,Frac RHK_FWY_deltazscores,Frac STNQCG_IVMALP_deltazscores,Frac STNQCG_FWY_deltazscores,Frac IVMALP_FWY_deltazscores,Frac WT_pos_block,Frac WT_neg_block,Frac WT_all_block,Frac Hydropathy,Frac NCPR
488,2069,4,0.3148,1.889,55.97,0.02868,0.903035,Q9BW19 Q86XP3 Q9NPA8 P28324,0.210934,0.75,...,0.25,0.0,0.0,0.0,0.0,0.0,0.25,0.25,0.0,0.25


In [916]:
[i for i in all_features if cluster_features_enrich.loc[488,'FDR '+i] < 0.05]

['Transcription factor e(y)2']

In [ ]:
[i for i in all_features if cluster_features_enrich.loc[488,'FDR '+i] < 0.05]

In [915]:
all_features = [i.replace('Frac ','') for i in cluster_features_enrich.columns if 'Frac' in i]

In [ ]:
len(all_features)

2695

## Feature that enriched in 493 clusters(domain, motif, aa features)

In [ ]:
sig_features = []
for f in cluster_features_enrich.columns:
    if 'FDR' in f:
        if len(cluster_features_enrich[cluster_features_enrich[f]<0.05]) > 0:
            sig_features.append(f.replace('FDR ',''))

In [ ]:
len(sig_features)

800

In [ ]:
frac_features = []
for f in cluster_features_enrich.columns:
    if 'Frac' in f:
        if len(cluster_features_enrich[cluster_features_enrich[f]>0.5]) > 0:
            frac_features.append(f.replace('Frac ',''))

In [ ]:
len(frac_features)

96

In [ ]:
feature_specific = [i for i in all_features if i in sig_features+frac_features]

In [ ]:
len(feature_specific)

810

In [ ]:
feature_specific_enrich_count = pd.DataFrame(index=feature_specific,columns = ['Count','Fraction','Clusters'])

for f in feature_specific_enrich_count.index:
    if 'FDR '+f in cluster_features_enrich.columns:
        f_rep1 = 'FDR '+f
        f_rep2 = 'Frac '+f
    else:
        f_rep1 = 'Cluster'
        f_rep2 = 'Cluster'
        
    if f_rep1 != 'Cluster':
        feature_specific_enrich_count.loc[f,'Count'] = len(cluster_features_enrich[(cluster_features_enrich[f_rep1]<0.05)|(cluster_features_enrich[f_rep2]>0.5)])
        feature_specific_enrich_count.loc[f,'Fraction'] = len(cluster_features_enrich[(cluster_features_enrich[f_rep1]<0.05)|(cluster_features_enrich[f_rep2]>0.5)])/len(cluster_features_enrich)
        feature_specific_enrich_count.loc[f,'Clusters'] = cluster_features_enrich[(cluster_features_enrich[f_rep1]<0.05)|(cluster_features_enrich[f_rep2]>0.5)]['Cluster'].tolist()
    else:
        feature_specific_enrich_count.loc[f,'Count'] = 0
        feature_specific_enrich_count.loc[f,'Fraction'] = 0
        feature_specific_enrich_count.loc[f,'Clusters'] = 0

In [ ]:
feature_consider = feature_specific_enrich_count[feature_specific_enrich_count['Count']>=5].iloc[1:]
feature_consider

,Count,Fraction,Clusters
14-3-3 protein,8,0.016227,"[100, 1632, 1734, 1811, 1890, 1895, 2032, 2040]"
AAA domain,7,0.014199,"[86, 171, 536, 785, 838, 1504, 1887]"
AF-4 proto-oncoprotein N-terminal region,5,0.010142,"[15, 32, 37, 482, 730]"
AF4 interaction motif,5,0.010142,"[15, 32, 37, 482, 730]"
"AFF4, C-terminal homology domain",5,0.010142,"[15, 32, 37, 482, 730]"
...,...,...,...
WT_pos_block,31,0.06288,"[1, 15, 18, 37, 44, 60, 67, 86, 92, 171, 277, ..."
WT_neg_block,37,0.075051,"[1, 13, 18, 44, 53, 67, 86, 92, 171, 277, 441,..."
WT_all_block,48,0.097363,"[1, 13, 15, 18, 22, 32, 34, 37, 44, 53, 67, 86..."
Hydropathy,8,0.016227,"[41, 119, 281, 1109, 1158, 1260, 1562, 1866]"


In [ ]:
list(feature_consider.index)

['14-3-3 protein',
 'AAA domain',
 'AF-4 proto-oncoprotein N-terminal region',
 'AF4 interaction motif',
 'AFF4, C-terminal homology domain',
 'ANC1 homology domain (AHD)',
 'Ankyrin repeats (3 copies)',
 'Armadillo/beta-catenin-like repeat',
 'Autism susceptibility gene 2 protein',
 'Bromodomain',
 'C-terminus of histone H2A',
 'C2 domain',
 'COMPASS (Complex proteins associated with Set1p) component N',
 'Cadherin domain',
 'Calponin homology (CH) domain',
 'Chromo (CHRromatin Organisation MOdifier) domain',
 'Collagen triple helix repeat (20 copies)',
 'Core histone H2A/H2B/H3/H4',
 'DEAD/DEAH box helicase',
 'Double-stranded RNA binding motif',
 'ELM2 domain',
 'FF domain',
 'Helicase conserved C-terminal domain',
 'Histidine phosphatase superfamily (branch 1)',
 'Histone chaperone Rttp106-like, middle domain',
 'Intermediate filament protein',
 'KH domain',
 'Keratin type II head',
 'LIM domain',
 'LSM domain',
 'MAPK-interacting and spindle-stabilising protein-like',
 'Myb-like D

In [ ]:

len(['14-3-3 protein',
 'AAA domain',
 'AF-4 proto-oncoprotein N-terminal region',
 'AF4 interaction motif',
 'AFF4, C-terminal homology domain',
 'ANC1 homology domain (AHD)',
 'Ankyrin repeats (3 copies)',
 'Armadillo/beta-catenin-like repeat',
 'Autism susceptibility gene 2 protein',
 'Bromodomain',
 'C-terminus of histone H2A',
 'C2 domain',
 'COMPASS (Complex proteins associated with Set1p) component N',
 'Cadherin domain',
 'Calponin homology (CH) domain',
 'Chromo (CHRromatin Organisation MOdifier) domain',
 'Collagen triple helix repeat (20 copies)',
 'Core histone H2A/H2B/H3/H4',
 'DEAD/DEAH box helicase',
 'Double-stranded RNA binding motif',
 'ELM2 domain',
 'FF domain',
 'Helicase conserved C-terminal domain',
 'Histidine phosphatase superfamily (branch 1)',
 'Histone chaperone Rttp106-like, middle domain',
 'Intermediate filament protein',
 'KH domain',
 'Keratin type II head',
 'LIM domain',
 'LSM domain',
 'MAPK-interacting and spindle-stabilising protein-like',
 'Myb-like DNA-binding domain',
 'Myosin head (motor domain)',
 'Myosin tail',
 'Novel AID APOBEC clade 2',
 'Occludin homology domain',
 'PBZ domain',
 'PDZ domain',
 'PH domain',
 'PHD-finger',
 'PHD-zinc-finger like domain',
 'Protein kinase domain',
 'RAWUL domain RING finger- and  WD40-associated ubiquitin-like',
 'RNA polymerase II elongation factor ELL',
 'RNA recognition motif',
 'Regulator of chromosome condensation (RCC1) repeat',
 'SAM domain (Sterile alpha motif)',
 'SAP domain',
 'SET domain',
 'SH2 domain',
 'SH3 domain',
 'Sec63 Brl domain',
 'Spectrin repeat',
 'Surp module',
 'Tubulin C-terminal domain',
 'Tubulin/FtsZ family, GTPase domain',
 'Variant SH3 domain',
 'WD domain, G-beta repeat',
 'WH1 domain',
 'WH2 motif',
 'YEATS family',
 'Yaf2/RYBP C-terminal binding motif',
 'Zinc finger C-x8-C-x5-C-x3-H type (and similar)',
 'Zinc finger, C2H2 type',
 'Zinc finger, C3HC4 type (RING finger)',
 'Zn-finger in Ran binding protein and others',
 'mbt repeat'])

67

In [ ]:
len(['CLV_PCSK_FUR_1',
 'CLV_PCSK_PC7_1',
 'DEG_Kelch_KLHL12_1',
 'DEG_Kelch_KLHL3_1',
 'DEG_SCF_COI1_1',
 'DOC_PP4_MxPP_1',
 'DOC_USP7_UBL2_3',
 'LIG_CNOT1_NIM_1',
 'LIG_EH_1',
 'LIG_EVH1_2',
 'LIG_HOMEOBOX',
 'LIG_IBS_1',
 'LIG_MYND_1',
 'LIG_PTAP_UEV_1',
 'LIG_RPA_C_Vert',
 'LIG_SH3_PxRPPK_7',
 'LIG_SH3_PxxDY_5',
 'LIG_SH3_PxxPPRxxK_8',
 'LIG_SH3_PxxxRxxKP_6',
 'LIG_TRAF6_MATH_1',
 'LIG_WW_1',
 'LIG_WW_2',
 'LIG_WW_3',
 'LIG_eIF4E_2',
 'MOD_CDC14_SPxK_1',
 'MOD_DYRK1A_RPxSP_1',
 'MOD_LATS_1',
 'MOD_PRMT_GGRGG_1',
 'TRG_DiLeu_BaEn_4'])

29

In [ ]:
len(['D',
 'E',
 'F',
 'G',
 'H',
 'K',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'Y',
 'acid',
 'basic',
 'hydrophilic',
 'aromatic',
 'charged',
 'nu',
 'betaTurn',
 'DE_omegazscores',
 'RHK_omegazscores',
 'DERHK_omegazscores',
 'STNQCG_omegazscores',
 'IVMALP_omegazscores',
 'NQ_omegazscores',
 'SG_omegazscores',
 'SR_omegazscores',
 'A_omegazscores',
 'D_omegazscores',
 'E_omegazscores',
 'G_omegazscores',
 'I_omegazscores',
 'K_omegazscores',
 'P_omegazscores',
 'Q_omegazscores',
 'R_omegazscores',
 'S_omegazscores',
 'T_omegazscores',
 'Y_omegazscores',
 'DE_RHK_deltazscores',
 'DE_STNQCG_deltazscores',
 'DE_IVMALP_deltazscores',
 'DE_FWY_deltazscores',
 'RHK_STNQCG_deltazscores',
 'RHK_IVMALP_deltazscores',
 'STNQCG_IVMALP_deltazscores',
 'IVMALP_FWY_deltazscores',
 'WT_pos_block',
 'WT_neg_block',
 'WT_all_block',
 'Hydropathy',
 'NCPR'])

52

### Clustering of condensates

In [ ]:
cluster_feature_enrich_df = pd.DataFrame(index=cluster_features_enrich['Cluster'].tolist(),columns=feature_consider.index)

In [ ]:
for i in feature_consider.index:
    for j in cluster_features_enrich['Cluster'].tolist():
        if j in feature_consider.loc[i,'Clusters']:
            cluster_feature_enrich_df.loc[j,i]=1
        else:
            cluster_feature_enrich_df.loc[j,i]=0

In [ ]:
cluster_feature_corr = cluster_feature_enrich_df.T.corr(method='spearman').fillna(0)

In [ ]:
cluster_feature_corr

,1,3,5,8,9,11,13,15,18,22,...,2056,2059,2064,2065,2068,2069,2078,2079,2082,2088
1,1.000000,0.030773,0.240772,0.128624,0.240772,-0.060193,0.342997,0.021339,0.685828,0.184637,...,0.416025,0.0,0.0,-0.060193,0.0,0.0,0.240772,-0.060193,-0.123091,0.342997
3,0.030773,1.000000,-0.029637,0.696631,-0.029637,-0.029637,-0.042220,0.089308,-0.026861,-0.060606,...,0.170697,0.0,0.0,-0.029637,0.0,0.0,-0.029637,-0.029637,0.204545,-0.042220
5,0.240772,-0.029637,1.000000,-0.020646,-0.014493,-0.014493,-0.020646,-0.046241,0.183892,-0.029637,...,-0.033389,0.0,0.0,-0.014493,0.0,0.0,-0.014493,-0.014493,-0.029637,-0.020646
8,0.128624,0.696631,-0.020646,1.000000,-0.020646,-0.020646,-0.029412,-0.065874,0.074848,-0.042220,...,0.285391,0.0,0.0,-0.020646,0.0,0.0,-0.020646,-0.020646,0.327205,-0.029412
9,0.240772,-0.029637,-0.014493,-0.020646,1.000000,-0.014493,-0.020646,-0.046241,0.183892,0.489010,...,-0.033389,0.0,0.0,-0.014493,0.0,0.0,-0.014493,-0.014493,-0.029637,-0.020646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2078,0.240772,-0.029637,-0.014493,-0.020646,-0.014493,-0.014493,-0.020646,0.313415,0.183892,-0.029637,...,-0.033389,0.0,0.0,-0.014493,0.0,0.0,1.000000,-0.014493,-0.029637,-0.020646
2079,-0.060193,-0.029637,-0.014493,-0.020646,-0.014493,-0.014493,-0.020646,-0.046241,0.183892,-0.029637,...,-0.033389,0.0,0.0,-0.014493,0.0,0.0,-0.014493,1.000000,-0.029637,-0.020646
2082,-0.123091,0.204545,-0.029637,0.327205,-0.029637,-0.029637,-0.042220,0.089308,-0.026861,-0.060606,...,-0.068279,0.0,0.0,-0.029637,0.0,0.0,-0.029637,-0.029637,1.000000,-0.042220


In [ ]:
cluster_feature_corr  = cluster_feature_enrich_df.T.corr().fillna(0)
cluster_feature_corr  = cluster_feature_corr.loc[:, (cluster_feature_corr != 0).any(axis=0)]
cluster_feature_corr  = cluster_feature_corr.loc[cluster_feature_corr.columns, :]

In [ ]:
len(cluster_feature_corr)

401

In [ ]:
cluster_feature_corr = cluster_feature_corr.loc[:, (cluster_feature_corr != 0).any(axis=0)]
cluster_feature_corr

,1,3,5,8,9,11,13,15,18,22,...,2002,2008,2015,2033,2056,2065,2078,2079,2082,2088
1,1.000000,0.030773,0.240772,0.128624,0.240772,-0.060193,0.342997,0.021339,0.685828,0.184637,...,-0.105802,-0.060193,0.234734,0.070535,0.416025,-0.060193,0.240772,-0.060193,-0.123091,0.342997
3,0.030773,1.000000,-0.029637,0.696631,-0.029637,-0.029637,-0.042220,0.089308,-0.026861,-0.060606,...,-0.052093,-0.029637,0.089308,-0.052093,0.170697,-0.029637,-0.029637,-0.029637,0.204545,-0.042220
5,0.240772,-0.029637,1.000000,-0.020646,-0.014493,-0.014493,-0.020646,-0.046241,0.183892,-0.029637,...,-0.025474,-0.014493,-0.046241,-0.025474,-0.033389,-0.014493,-0.014493,-0.014493,-0.029637,-0.020646
8,0.128624,0.696631,-0.020646,1.000000,-0.020646,-0.020646,-0.029412,-0.065874,0.074848,-0.042220,...,-0.036290,-0.020646,0.190304,-0.036290,0.285391,-0.020646,-0.020646,-0.020646,0.327205,-0.029412
9,0.240772,-0.029637,-0.014493,-0.020646,1.000000,-0.014493,-0.020646,-0.046241,0.183892,0.489010,...,-0.025474,-0.014493,-0.046241,-0.025474,-0.033389,-0.014493,-0.014493,-0.014493,-0.029637,-0.020646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2078,0.240772,-0.029637,-0.014493,-0.020646,-0.014493,-0.014493,-0.020646,0.313415,0.183892,-0.029637,...,-0.025474,-0.014493,0.313415,-0.025474,-0.033389,-0.014493,1.000000,-0.014493,-0.029637,-0.020646
2079,-0.060193,-0.029637,-0.014493,-0.020646,-0.014493,-0.014493,-0.020646,-0.046241,0.183892,-0.029637,...,-0.025474,-0.014493,-0.046241,-0.025474,-0.033389,-0.014493,-0.014493,1.000000,-0.029637,-0.020646
2082,-0.123091,0.204545,-0.029637,0.327205,-0.029637,-0.029637,-0.042220,0.089308,-0.026861,-0.060606,...,0.251784,-0.029637,0.089308,0.251784,-0.068279,-0.029637,-0.029637,-0.029637,1.000000,-0.042220


In [ ]:
from sklearn.cluster import AgglomerativeClustering

for i in range(20,31):
    model = AgglomerativeClustering(n_clusters=i, 
                                    #metric='minkowski',
                                    #metric='cosine',
                                    #affinity='braycurtis',
                                    #metric='canberra',
                                    #affinity='chebyshev',
                                    #affinity='cityblock',
                                    #metric='correlation',
                                    #affinity='seuclidean',
                                    #affinity='sqeuclidean',
                                    #metric='hamming',
                                    #affinity='kulsinski',
                                    #metric='euclidean',
                                    #affinity='dice',
                                     memory=None, connectivity=None,
                                    compute_full_tree='auto', 
                                    #linkage='ward')
                                    linkage='complete')
                                    #linkage='average')
                                    #linkage='single')

    #labels = model.fit_predict(cluster_feature_enrich_df_plot.loc[:,feature_specific].fillna(0).values)
    #cluster_feature_enrich_df_plot[f'Complete_{i}'] = labels
#euclidean

    labels = model.fit_predict(cluster_feature_corr.loc[:,cluster_feature_corr.index].values)
    cluster_feature_corr[f'Complete_{i}'] = labels

In [ ]:
cluster_feature_corr_color = [colors_dict[i] for i in cluster_feature_corr['Complete_27'].tolist()]
cluster_feature_corr_color

['#496E93',
 '#789C49',
 '#F4C8C5',
 'palegreen',
 '#007440',
 '#955B8E',
 '#496E93',
 'black',
 '#47499C',
 '#496E93',
 '#5AA987',
 '#ACC532',
 '#5AA987',
 '#789C49',
 '#ACC532',
 'black',
 '#789C49',
 '#496E93',
 'black',
 '#AA97BC',
 '#7D89C1',
 '#F5D19E',
 '#E48180',
 '#F5D19E',
 'darkorange',
 '#F4C8C5',
 '#ACC532',
 '#F5D19E',
 '#007440',
 '#007440',
 '#8080EB',
 '#496E93',
 '#47499C',
 '#5AA987',
 '#8080EB',
 '#007440',
 '#919191',
 '#8080EB',
 '#496E93',
 'darkorange',
 '#E48180',
 '#F5D19E',
 '#8080EB',
 '#8080EB',
 '#ACC532',
 '#E8DB00',
 '#F4C8C5',
 '#789C49',
 '#E48180',
 '#BFCBF1',
 '#007440',
 '#919191',
 '#ACC532',
 '#007440',
 '#9DCAE6',
 '#B8C98D',
 '#8080EB',
 '#F6E2DC',
 '#789C49',
 '#E8DB00',
 '#C24F4F',
 '#F6E2DC',
 '#C24F4F',
 '#F6E2DC',
 '#8080EB',
 '#B8C98D',
 '#5AA987',
 '#8080EB',
 '#F5D19E',
 '#47499C',
 '#93583A',
 '#F5D19E',
 '#7D89C1',
 '#789C49',
 '#BFCBF1',
 'palegreen',
 '#F5D19E',
 '#ACC532',
 '#AA97BC',
 '#BFCBF1',
 '#47499C',
 '#522B55',
 '#F5D19E',


In [ ]:
sns.clustermap(
    cluster_feature_corr.loc[:,cluster_feature_corr.index],
    #cluster_feature_enrich_df_plot.loc[:,feature_specific].fillna(0),
#               vmin=-1,vmax=1,center=0,           
              vmin=-0.5,vmax=0.5,center=0,
               cmap=my_colormap1 ,
               row_colors = cluster_feature_corr_color,
               #linewidths=0.8,
               linecolor='white',
               #method='ward', 
               method='complete', 
               #method='average',
               #col_cluster=False,
               #method='single',
               
               #metric='euclidean'
               #metric='sqeuclidean'
               #metric='seuclidean'
               #metric='cosine',
               #metric='minkowski'
               #metric='braycurtis'
               #metric='canberra',
               #metric='chebyshev'
               #metric='cityblock'
               #metric='correlation',
               #metric='hamming'
               #metric='kulsinski'
               #metric='dice',
               figsize=(30,30)

               )


d:\conda\Lib\site-packages\seaborn\matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)
d:\conda\Lib\site-packages\seaborn\matrix.py:560: UserWarning: Clustering large matrix with scipy. Installing `fastcluster` may give better performance.
  warnings.warn(msg)


In [ ]:
condensate_type_dict = cluster_feature_corr.to_dict()[f'Complete_27']
cluster_feature_enrich_df[f'type_27'] = cluster_feature_enrich_df.index.map(condensate_type_dict)

In [ ]:
cluster_feature_enrich_df['cluster'] = cluster_feature_enrich_df.index

In [ ]:
cluster_feature_enrich_df.index = cluster_features_enrich['Label'].tolist()